In [ ]:
! pip install numpy torch blobfile tqdm pyYaml pillow mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 73 kB 764 kB/s 
     |████████████████████████████████| 2.5 MB 17.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.3 MB 25.3 MB/s 
     |████████████████████████████████| 140 kB 69.8 MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp37-cp37m-linux_x86_64.whl size=2185769 sha256=0fd8787950dcc715a5cb01ddc4e3b5313f68f80d7ef61a3da86112ef02a2a622
  Stored in directory: /root/.cache/pip/wheels/99/54/29/187b5768bbb7beeab6753bc30acf56f35bc8ca9c214a31e173
Successfully built mpi4py
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
REPAINT_PATH = "/content/drive/MyDrive/Shadow\ Removal\ DDPM\ Project/RePaint/test.py"
CONF_PATH = "/content/drive/MyDrive/Shadow\ Removal\ DDPM\ Project/RePaint/confs/face_example.yml"

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
!python $REPAINT_PATH --conf_path $CONF_PATH

Start face_example
cosine
4000
---------
[9.86581881e-06 1.01694149e-05 1.04730140e-05 ... 5.55555443e-01
 7.49999962e-01 9.99000000e-01]
sampling...
100% 7168/7168 [2:38:20<00:00,  1.33s/it]
 33% 2400/7168 [53:02<2:09:08,  1.63s/it]

In [ ]:
import numpy as np
import torch

In [ ]:
def get_schedule_jump(t_T, n_sample, jump_length, jump_n_sample,
                      jump2_length=1, jump2_n_sample=1,
                      jump3_length=1, jump3_n_sample=1,
                      start_resampling=100000000):

    jumps = {}
    for j in range(0, t_T - jump_length, jump_length):
        jumps[j] = jump_n_sample - 1

    jumps2 = {}
    for j in range(0, t_T - jump2_length, jump2_length):
        jumps2[j] = jump2_n_sample - 1

    jumps3 = {}
    for j in range(0, t_T - jump3_length, jump3_length):
        jumps3[j] = jump3_n_sample - 1

    t = t_T
    ts = []

    while t >= 1:
        t = t-1
        ts.append(t)

        if (
            t + 1 < t_T - 1 and
            t <= start_resampling
        ):
            for _ in range(n_sample - 1):
                t = t + 1
                ts.append(t)

                if t >= 0:
                    t = t - 1
                    ts.append(t)

        if (
            jumps3.get(t, 0) > 0 and
            t <= start_resampling - jump3_length
        ):
            jumps3[t] = jumps3[t] - 1
            for _ in range(jump3_length):
                t = t + 1
                ts.append(t)

        if (
            jumps2.get(t, 0) > 0 and
            t <= start_resampling - jump2_length
        ):
            jumps2[t] = jumps2[t] - 1
            for _ in range(jump2_length):
                t = t + 1
                ts.append(t)
            jumps3 = {}
            for j in range(0, t_T - jump3_length, jump3_length):
                jumps3[j] = jump3_n_sample - 1

        if (
            jumps.get(t, 0) > 0 and
            t <= start_resampling - jump_length
        ):
            jumps[t] = jumps[t] - 1
            for _ in range(jump_length):
                t = t + 1
                ts.append(t)
            jumps2 = {}
            for j in range(0, t_T - jump2_length, jump2_length):
                jumps2[j] = jump2_n_sample - 1

            jumps3 = {}
            for j in range(0, t_T - jump3_length, jump3_length):
                jumps3[j] = jump3_n_sample - 1

    ts.append(-1)

    return ts

In [1]:
!pip install lpips

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.7 MB/s 


In [36]:
shadow_results = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test/test_results"
shadow_free = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test/test_non_shadow"
original_test = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test_backup/test_C"
resized = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test/non_shadow_resized/"
decay_results = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test/decay_results/"
decay_original = "/content/drive/MyDrive/Shadow_Removal_DDPM_Project/ISTD_Dataset/test/decay_original/"

import cv2

In [22]:
import lpips
import os

loss_fn = lpips.LPIPS(net='alex')
gt = os.listdir(resized)
res = os.listdir(shadow_results)

print(len(gt))
print(len(res))

# d = loss_fn.forward(im0,im1)
# for i in range(len(gt)):
  # print(gt[i], res[i])


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
51
51


In [23]:
# shadow_results_files = os.listdir(shadow_results)
# for i in range(len(shadow_results_files)):
#   # print(shadow_results_files[i])
#   img = cv2.imread(os.path.join(original_test,shadow_results_files[i]))
#   img_resize = cv2.resize(img, (256,256))
#   print(os.path.join(resized, shadow_results_files[i]))
#   # cv2.imwrite(os.path.join(resized, shadow_results_files[i]), img_resize)
#   # print(img.shape)


In [28]:
import torch
import lpips
from IPython import embed

use_gpu = False         # Whether to use GPU
spatial = True         # Return a spatial map of perceptual distance.

# Linearly calibrated models (LPIPS)
loss_fn = lpips.LPIPS(net='alex', spatial=spatial) # Can also set net = 'squeeze' or 'vgg'
# loss_fn = lpips.LPIPS(net='alex', spatial=spatial, lpips=False) # Can also set net = 'squeeze' or 'vgg'

if(use_gpu):
	loss_fn.cuda()

## Example usage with dummy tensors
# dummy_im0 = torch.zeros(1,3,64,64) # image should be RGB, normalized to [-1,1]
# dummy_im1 = torch.zeros(1,3,64,64)
# if(use_gpu):
# 	dummy_im0 = dummy_im0.cuda()
# 	dummy_im1 = dummy_im1.cuda()
# dist = loss_fn.forward(dummy_im0,dummy_im1)

## Example usage with images
distance = []
for i in range(len(gt)):

  ex_ref = lpips.im2tensor(lpips.load_image(os.path.join(resized, gt[i])))
  ex_p0 = lpips.im2tensor(lpips.load_image(os.path.join(shadow_results, gt[i])))
  # ex_p1 = lpips.im2tensor(lpips.load_image('./imgs/ex_p1.png'))
  if(use_gpu):
    ex_ref = ex_ref.cuda()
    ex_p0 = ex_p0.cuda()
    # ex_p1 = ex_p1.cuda()

  ex_d0 = loss_fn.forward(ex_ref,ex_p0)
  # ex_d1 = loss_fn.forward(ex_ref,ex_p1)

  if not spatial:
      print('Distances: (%.3f)'%(ex_d0))
      distance.append(ex_d0)
  else:
      print('Distances: (%.3f)'%(ex_d0.mean()))            # The mean distance is approximately the same as the non-spatial distance
      distance.append(ex_d0.mean())
      # Visualize a spatially-varying distance map between ex_p0 and ex_ref
      # import pylab
      # pylab.imshow(ex_d0[0,0,...].data.cpu().numpy())
      # pylab.show()

print("LPIPS : ", sum(distance)/len(distance))

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Distances: (0.315)
Distances: (0.356)
Distances: (0.415)
Distances: (0.329)
Distances: (0.308)
Distances: (0.479)
Distances: (0.356)
Distances: (0.302)
Distances: (0.418)
Distances: (0.432)
Distances: (0.289)
Distances: (0.357)
Distances: (0.493)
Distances: (0.281)
Distances: (0.287)
Distances: (0.491)
Distances: (0.281)
Distances: (0.223)
Distances: (0.319)
Distances: (0.303)
Distances: (0.279)
Distances: (0.468)
Distances: (0.309)
Distances: (0.203)
Distances: (0.464)
Distances: (0.148)
Distances: (0.365)
Distances: (0.423)
Distances: (0.351)
Distances: (0.257)
Distances: (0.302)
Distances: (0.266)
Distances: (0.240)
Distances: (0.271)
Distances: (0.289)
Distances: (0.247)
Distances: (0.293)
Distances: (0.245)
Distances: (0.332)
Distances: (0.387)
Distances: (0.331)
Distances: (0.265)
Distances: (0.190)
Distances: (0.188)
Distances: (0.413)
Distances: (0.238)
Distances: (0.350)
Distances: (0.278)
D

In [39]:
decay_results_files = os.listdir(decay_results)
decay_original_files = os.listdir(decay_original)
for i in range(len(decay_results_files)):
  # print(decay_results_files[i])
  # img = cv2.imread(os.path.join(original_test,decay_results_files[i]))
  # print(img.shape)
  # img_resize = cv2.resize(img, (256,256))
  # print(os.path.join(decay_original, decay_results_files[i]))
  # cv2.imwrite(os.path.join(decay_original, decay_results_files[i]), img_resize)
  # print(img.shape)
  print(decay_results_files[i], decay_original_files[i])

121-3.png 121-3.png
109-11.png 109-11.png
115-5.png 115-5.png
117-17.png 117-17.png
117-5.png 117-5.png
112-4.png 112-4.png
114-10.png 114-10.png
108-10.png 108-10.png
124-2.png 124-2.png
93-3.png 93-3.png
105-19.png 105-19.png
104-1.png 104-1.png
101-4.png 101-4.png
91-10.png 91-10.png
91-4.png 91-4.png
120-7.png 120-7.png
96-1.png 96-1.png


In [41]:
import torch
import lpips
from IPython import embed

use_gpu = False         # Whether to use GPU
spatial = True         # Return a spatial map of perceptual distance.

# Linearly calibrated models (LPIPS)
loss_fn = lpips.LPIPS(net='alex', spatial=spatial) # Can also set net = 'squeeze' or 'vgg'
# loss_fn = lpips.LPIPS(net='alex', spatial=spatial, lpips=False) # Can also set net = 'squeeze' or 'vgg'

if(use_gpu):
	loss_fn.cuda()

## Example usage with dummy tensors
# dummy_im0 = torch.zeros(1,3,64,64) # image should be RGB, normalized to [-1,1]
# dummy_im1 = torch.zeros(1,3,64,64)
# if(use_gpu):
# 	dummy_im0 = dummy_im0.cuda()
# 	dummy_im1 = dummy_im1.cuda()
# dist = loss_fn.forward(dummy_im0,dummy_im1)

## Example usage with images
distance = []
for i in range(len(decay_results_files)):

  ex_ref = lpips.im2tensor(lpips.load_image(os.path.join(decay_results, decay_results_files[i])))
  ex_p0 = lpips.im2tensor(lpips.load_image(os.path.join(decay_original, decay_results_files[i])))
  # ex_p1 = lpips.im2tensor(lpips.load_image('./imgs/ex_p1.png'))
  if(use_gpu):
    ex_ref = ex_ref.cuda()
    ex_p0 = ex_p0.cuda()
    # ex_p1 = ex_p1.cuda()

  ex_d0 = loss_fn.forward(ex_ref,ex_p0)
  # ex_d1 = loss_fn.forward(ex_ref,ex_p1)

  if not spatial:
      print('Distances: (%.3f)'%(ex_d0))
      distance.append(ex_d0)
  else:
      print('Distances: (%.3f)'%(ex_d0.mean()))            # The mean distance is approximately the same as the non-spatial distance
      distance.append(ex_d0.mean())
      # Visualize a spatially-varying distance map between ex_p0 and ex_ref
      # import pylab
      # pylab.imshow(ex_d0[0,0,...].data.cpu().numpy())
      # pylab.show()

print("LPIPS : ", sum(distance)/len(distance))

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Distances: (0.464)
Distances: (0.225)
Distances: (0.309)
Distances: (0.281)
Distances: (0.466)
Distances: (0.307)
Distances: (0.351)
Distances: (0.276)
Distances: (0.261)
Distances: (0.288)
Distances: (0.201)
Distances: (0.283)
Distances: (0.254)
Distances: (0.279)
Distances: (0.363)
Distances: (0.504)
Distances: (0.149)
LPIPS :  tensor(0.3094, grad_fn=<DivBackward0>)
